# CAN-DB 관리

In [ ]:
# HOW TO USE
# 자체제어기 Excel 테이블을 클립보드로 복사(Ctrl+C) 한 뒤 아래 코드를 수행
# 기본 저장 경로: SVN 내 CAN 관리 폴더
# @saveAs: 저장할 경로 선택
from pyems.candb.vcs import CanDbVersionControl
vcs = CanDbVersionControl()
vcs.clipbd2db()

# 자동화 모델 생성: %ComDef *_HEV

In [ ]:
# HOW TO USE
# @engine_spec 결정: "ICE" / "HEV"
# (필요 시) 제외 메시지에 대한 Customize
from pyems.candb.reader import CanDb
from cannect.can.ascet.ComDef import ComDef

db = CanDb()
engine_spec = "ICE"

# DB CUSTOMIZE ------------------------------------------------------
exclude_ecus = ["EMS", "CVVD", "MHSG", "NOx"]
if engine_spec == "ICE":
    exclude_ecus += ["BMS", "LDC"]
db = db[~db["ECU"].isin(exclude_ecus)]

# db = db[db["Status"] != "TSW"] # TSW 제외
# db = db[~db["Requirement ID"].isin(["VCDM CR10777888"])] # 특정 CR 제외
# db = db[~db["Required Date"].isin(["2024-08-27"])] # 특정 일자 제외
db = db[~db["Message"].isin([ # 특정 메시지 제외
    "ADAS_UX_02_50ms",
    "HU_CLU_USM_01_00ms",
    "HU_CLU_USM_E_01",
    "HU_NAVI_05_200ms",
    "HU_NAVI_V2_3_POS_PE",
    "HU_CLOCK_01_1000ms",
    "HU_CLOCK_PE_02",
])]
# db.revision = "TEST SW" # 공식SW는 주석 처리
# DB CUSTOMIZE END --------------------------------------------------
db = db.to_developer_mode(engine_spec)

model = ComDef(
    db=db,
    engine_spec=engine_spec,
    base_model="", # 미 입력 시, SVN 최신 모델 사용
    exclude_tsw=True,
)
model.autorun()

# 자동화 모델 생성: %ComRx *_HEV

# 자동화 모델 생성: 진단 모델

# TEST CASE

## TEST REPORT 생성(Excel)

## XDA 생성

### [XDA 생성] 진단

In [4]:
from cannect.can.preset import DIAGNOSIS_ICE
from cannect.can.testcase.xda.diagnosis import XdaDiagnosis
import os

root = r'\\kefico\keti\ENT\Softroom\Temp\J.H.Lee\00 CR\CR10785931 J1979-2 CAN 진단 대응 ICE CANFD\08_Verification'
for model, messages in DIAGNOSIS_ICE.items():
    dat = os.path.join(root, rf'Data/{model}_Det_Diag.dat')
    for message in messages:
        try:
            print(model, message, end=": ")
            xda = XdaDiagnosis(dat=dat, message=message)
            xda.create(os.path.join(root, 'xda'))
            print("SUCCESS")
        except Exception as e:
            print("FAILED", e)

CanFDABSD ABS_ESC_01_10ms: SUCCESS
CanFDABSD WHL_01_10ms: SUCCESS
CanFDACUD ACU_01_100ms: SUCCESS
CanFDACUD IMU_01_10ms: SUCCESS
CanFDADASD ADAS_CMD_10_20ms: SUCCESS
CanFDADASD ADAS_CMD_20_20ms: SUCCESS
CanFDADASD ADAS_PRK_20_20ms: SUCCESS
CanFDADASD ADAS_PRK_21_20ms: SUCCESS
CanFDBCMD BCM_02_200ms: SUCCESS
CanFDBCMD BCM_07_200ms: SUCCESS
CanFDBCMD BCM_10_200ms: SUCCESS
CanFDBCMD BCM_20_200ms: SUCCESS
CanFDBCMD BCM_22_200ms: SUCCESS
CanFDBDCD BDC_FD_05_200ms: SUCCESS
CanFDBDCD BDC_FD_07_200ms: SUCCESS
CanFDBDCD BDC_FD_08_200ms: SUCCESS
CanFDBDCD BDC_FD_10_200ms: SUCCESS
CanFDBDCD BDC_FD_SMK_02_200ms: SUCCESS
CanBMSD_48V BMS5: SUCCESS
CanBMSD_48V BMS6: SUCCESS
CanBMSD_48V BMS7: SUCCESS
CanFDCCUD CCU_OBM_01_1000ms: SUCCESS
CanFDCCUD CCU_OTA_01_200ms: SUCCESS
CanFDCLUD CLU_01_20ms: SUCCESS
CanFDCLUD CLU_02_100ms: SUCCESS
CanFDCLUD CLU_18_20ms: SUCCESS
CanCVVDD CVVD1: FAILED Can not find file \\kefico\keti\ENT\Softroom\Temp\J.H.Lee\00 CR\CR10785931 J1979-2 CAN 진단 대응 ICE CANFD\08_Verificati